In [5]:
### LLM(Large language model) 호출
from langchain.llms import OpenAI # text-davinci-003
from langchain.chat_models import ChatOpenAI # gpt-3.5-turbo

# llm = OpenAI()
# result = llm.predict("태양계 행성은 총 몇개가 있어?")

# temperature=> 얼마나 창의적 + 무작위적 으로 하는가에 대한 수치 높을 수록 더 창의적 이고 무작위 적으로 생성됨.
chat = ChatOpenAI(temperature=0.1)
result = chat.predict("태양계 행성은 총 몇개가 있어?")

result

'태양계 행성은 총 8개가 있습니다. 이는 수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성입니다.하지만 명왕성은 2006년에 왜곡체로 지정되어 행성에서 제외되었습니다.'

In [8]:
# Predict Messages
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="당신은 지리 전문가입니다. 그리고 당신은 한글으로만 답변합니다."),
    AIMessage(content="안녕하세요, 제 이름은 파올로입니다!"),
    HumanMessage(content="멕시코와 태국 사이의 거리는 어느정도 되나요? 그리고 당신의 이름은 무엇인가요?")
]
chat.predict_messages(messages)


AIMessage(content='멕시코와 태국 사이의 직선 거리는 약 15,000km 정도입니다. 제 이름은 파올로입니다.')